In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random

In [ ]:
data_dir = '../input/chinese-mnist/data/data'

In [ ]:
mnist=pd.read_csv('../input/chinese-mnist/chinese_mnist.csv')
mnist

In [ ]:
Name=sorted(mnist['value'].unique())
Name

In [ ]:
n=len(Name)
print(n)

N=[]
for i in range(n):
    N+=[i]
    
normal_mapping=dict(zip(Name,N )) 
reverse_mapping=dict(zip(N,Name)) 

def mapper(value):
    return reverse_mapping[value]

print(normal_mapping)

In [ ]:
datax0=[]
dataf0=[]
for im in tqdm(os.listdir(data_dir)):
    image=load_img(os.path.join(data_dir,im), grayscale=False, color_mode='rgb', target_size=(50,50))
    image=img_to_array(image)
    image=image/255.0
    datax0+=[image]
    dataf0+=[im]

In [ ]:
dataf1=[]
for item in dataf0:
    dataf1+=[int(item[0:-4].split('_')[3])-1]

In [ ]:
m=len(datax0)
M=[]
for i in range(m):
    M+=[i]
random.shuffle(M)

In [ ]:
datax2=np.array(datax0)
dataf2=np.array(dataf1)
trainx0=datax2[M[0:(m//4)*3]]
testx0=datax2[M[(m//4)*3:]]
trainy0=dataf2[M[0:(m//4)*3]]
testy0=dataf2[M[(m//4)*3:]]

In [ ]:
trainy1=to_categorical(trainy0)
#testy1=to_categorical(testy0)

In [ ]:
Trainx1=np.array(trainx0)
Trainy1=np.array(trainy1)
Testx1=np.array(testx0)
#Testy1=np.array(testy1)

In [ ]:
trainx,testx,trainy,testy=train_test_split(Trainx1,Trainy1,test_size=0.2,random_state=44)

In [ ]:
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
                        width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

In [ ]:
pretrained_model3 = tf.keras.applications.DenseNet201(input_shape=(50,50,3),include_top=False,weights='imagenet',pooling='avg')
pretrained_model3.trainable = False

In [ ]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(15, activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
his=model.fit(datagen.flow(trainx,trainy,batch_size=32),validation_data=(testx,testy),epochs=40)

In [ ]:
y_pred=model.predict(testx)
pred=np.argmax(y_pred,axis=1)
ground = np.argmax(testy,axis=1)
print(classification_report(ground,pred))

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
img0='../input/chinese-mnist/data/data/input_100_1_13.jpg'
load_img(img0,target_size=(200,200))

In [ ]:
image=load_img(img0,target_size=(50,50))
image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image=np.expand_dims(image, axis=0)

In [ ]:
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
pred_name=mapper(value)
name0=img0[0:-4].split('_')[3]
name=mapper(int(name0)-1)
print("Prediction is {}.".format(pred_name))
print("Answer is {}.".format(name))

In [ ]:
pred2=model.predict(Testx1)
print(Testx1.shape)
print(pred2.shape)

PRED=[]
for item in pred2:
    value2=np.argmax(item)      
    PRED+=[value2]
    
ANS=testy0

In [ ]:
print(PRED[0:10])
print(ANS[0:10])

In [ ]:
accuracy=accuracy_score(ANS,PRED)
print(accuracy)